# INDENTATION 2D EXAMPLE WITH ARGIOPE AND HARDNESS 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import hardness as hd
import argiope as ag
import pandas as pd
import numpy as np
import os, subprocess, time, local_settings, time
%matplotlib nbagg

# USEFUL FUNCTIONS
def create_dir(path):
  try:
    os.mkdir(path)
  except:
    pass

AttributeError: module 'argiope' has no attribute 'utils'

## Settings

In [ ]:
# SETTINGS
workdir   = "workdir/"
outputdir = "outputs/"
label   = "indentation_2D"

create_dir(workdir)
create_dir(workdir + outputdir)     

## Model definition

In [ ]:
#-------------------------------------------------------------------------------
# MESH DEFINITIONS
def element_map(mesh):
    mesh.elements.loc[mesh.elements.type.argiope == "tri3", ("type", "solver", "")] = "CAX3" 
    mesh.elements.loc[mesh.elements.type.argiope == "quad4", ("type", "solver", "")] = "CAX4" 
    return mesh
    
    
def sample_material_map(mesh):
    mesh.elements["materials"] = "SAMPLE_MAT" 
    return mesh

def indenter_material_map(mesh):
    mesh.elements["materials"] = "INDENTER_MAT" 
    return mesh
    
    
meshes = {
    "sample" : hd.models.sample_mesh_2D("gmsh", 
                                   workdir, 
                                   lx = 1., ly = 1., 
                                   r1 = 2., r2 = 4., 
                                   Nx = 32, Ny = 32, 
                                   Nr = 4, Nt = 64,
                                   element_map = element_map,
                                   material_map = sample_material_map),
                                   
    "indenter" : hd.models.spheroconical_indenter_mesh_2D("gmsh", 
                                   workdir, 
                                   R = 1.,
                                   psi= 30., 
                                   r1 = 1., 
                                   r2 = 10., 
                                   r3 = 10., 
                                   lc1 = .03, 
                                   lc2 = 20.,
                                   rigid = False,
                                   element_map = element_map,
                                   material_map = indenter_material_map)}
                                   
materials = [ag.materials.Hollomon(label = "SAMPLE_MAT", strain_data_points = 10),
             ag.materials.Hollomon(label = "INDENTER_MAT", strain_data_points = 10)]
    
"""
sample_mesh.save(h5path = workdir + outputdir + simName + "_sample_mesh.h5")
indenter_mesh.save(h5path = workdir + outputdir + simName + "_indenter_mesh.h5")  
"""   
#-------------------------------------------------------------------------------
# STEP DEFINTIONS
steps = [
        hd.models.indentation_2D_step_input(name = "LOADING1",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.1),
        hd.models.indentation_2D_step_input(name = "UNLOADING1",
                                            control_type = "force", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = 0.),
        hd.models.indentation_2D_step_input(name = "RELOADING1",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.1),
        hd.models.indentation_2D_step_input(name = "LOADING2",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.2),                                    
        hd.models.indentation_2D_step_input(name = "UNLOADING2",
                                            control_type = "force",
                                            kind = "adaptative", 
                                            duration = 1., 
                                            nframes = 50,
                                            controlled_value = 0.)
        ]                                                                                                  
#-------------------------------------------------------------------------------
"""                                  
hd.models.indentation_2D_input(sample_mesh   = sample_mesh,
                            indenter_mesh = indenter_mesh,
                            steps = steps ,
                            path = workdir + simName + ".inp")
                                     
hd.postproc.indentation_abqpostproc(path = workdir + simName + "_abqpp.py")        
hd.postproc.indentation_pypostproc(path = workdir + simName + "_pypp.py") 
"""
#-------------------------------------------------------------------------------
model = hd.models.Indentation2D(label = label, 
                      meshes = meshes, 
                      steps = steps, 
                      materials = materials, 
                      solver = "abaqus", 
                      solver_path = local_settings.ABAQUS_PATH,
                      workdir = workdir,
                      verbose = True)



In [ ]:
model.write_input()
#model.run_simulation()
model.write_postproc()
model.run_postproc()


In [ ]:
#model.write_postproc()
#model.run_postproc()
model.save(workdir + label + ".pcklz")
model = ag.utils.load(workdir + label + ".pcklz")

In [ ]:
path = "workdir/reports/indentation_2D_instance-I_INDENTER_step-LOADING1_frame--1_var-S.frpt"

In [ ]:
mdata, data = ag.abq.pypostproc.read_field_report(path)

In [ ]:
indenter =  hd.models.SpheroconicalIndenter2D(
                                   R = 1.,
                                   psi= 30., 
                                   r1 = 1., 
                                   r2 = 10., 
                                   r3 = 10., 
                                   lc1 = .03, 
                                   lc2 = 20.,
                                   rigid = False,
                                   gmsh_path = "gmsh",
                                   file_name = "dummy", 
                                   workdir = workdir, 
                                   gmsh_space = 2, 
                                   gmsh_options = "-algo 'delquad'",
                                   element_map = element_map,
                                   material_map = indenter_material_map)

indenter.make_mesh()
indenter.mesh.elements.head()

In [ ]:
hd.models.SpheroconicalIndenter2D()

In [4]:
import argiope as ag
dir(ag)

AttributeError: module 'argiope' has no attribute 'utils'

In [ ]:
ag.mesh

In [5]:
import argiope.utils

AttributeError: module 'argiope' has no attribute 'utils'